# Bitwise AND operation

In [ ]:
# Constants
N = 2
SHOTS = 10

In [ ]:
# Imports
from itertools import product
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import AndGate
from qiskit.primitives import StatevectorSampler

In [ ]:
# Create registers
a = QuantumRegister(N, 'a')       # First input
b = QuantumRegister(N, 'b')       # Second input
target = QuantumRegister(N, 't')  # Targets to store AND results
res = ClassicalRegister(N, 'res')

sampler = StatevectorSampler()

In [ ]:
def generate_circuit(a, b, target, res, a_val, b_val, ANDFunc):
    # Create the quantum circuit
    qc = QuantumCircuit(a, b, target, res)

    # Initialize the input states
    state_a = Statevector.from_int(a_val, dims=2**N)
    qc.initialize(state_a.data, a)
    state_b = Statevector.from_int(b_val, dims=2**N)
    qc.initialize(state_b.data, b)

    # Apply the AND gate to each pair of qubits
    for i in range(N):
        ANDFunc(qc, a[i], b[i], target[i])
    
    qc.measure(target, res)

    return qc

## Qiskit Library usage

In [ ]:
and_gate = AndGate(2) # AND gate takes 2 qubits as input

def QiskitAND(qc, a, b, target):
    qc.append(and_gate, [a, b, target])

qc = generate_circuit(a, b, target, res, 0, 0, QiskitAND)
qc.draw('mpl')

In [ ]:
# Generate all combinations of inputs for A and B and run the AND circuit for each combination
for a_val, b_val in product(range(2**N), range(2**N)):
    qc = generate_circuit(a, b, target, res, a_val, b_val, QiskitAND)
    
    result = sampler.run([qc], shots=SHOTS).result()
    print(f"Input A: {a_val:04b}, Input B: {b_val:04b}, Output AND: {result[0].data.res.get_counts()}\n")


## Custom Implementation

### Running on qubits in their eigenstates

#### First, I will use Qiskit's provided Toffoli (CCX) gate to implement the AND functionality

In [ ]:
def ToffoliGateAND(qc, a, b, target):
    qc.ccx(a, b, target)

qc = generate_circuit(a, b, target, res, 0, 0, ToffoliGateAND)
qc.draw('mpl')


In [ ]:
for a_val, b_val in product(range(2**N), range(2**N)):
    qc = generate_circuit(a, b, target, res, a_val, b_val, QiskitAND)
    
    result = sampler.run([qc], shots=SHOTS).result()
    print(f"Input A: {a_val:04b}, Input B: {b_val:04b}, Output AND: {result[0].data.res.get_counts()}\n")

#### Now, I will implement it from scratch

In [ ]:
def ElementaryGatesAND(qc, a, b, target):
    qc.h(target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.t(target)
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.t(target)
    qc.h(target)

qc = generate_circuit(a, b, target, res, 0, 0, ElementaryGatesAND)
qc.draw('mpl')

In [ ]:
for a_val, b_val in product(range(2**N), range(2**N)):
    qc = generate_circuit(a, b, target, res, a_val, b_val, ElementaryGatesAND)
    
    result = sampler.run([qc], shots=SHOTS).result()
    print(f"Input A: {a_val:04b}, Input B: {b_val:04b}, Output AND: {result[0].data.res.get_counts()}\n")

## Running on qubits in superposition